<a href="https://colab.research.google.com/github/ykato27/GAN/blob/main/StyleGAN2_ADA_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN2-ADA-PyTorch

**Notes**
* Training and Inference sections should be fairly stable. I’ll slowly add new features but it should work for most mainstream use cases.
* Advanced Features are being documented toward the bottom of this notebook

---

If you find this notebook useful, consider signing up for my [Patreon](https://www.patreon.com/bustbright) or [YouTube channel](https://www.youtube.com/channel/UCaZuPdmZ380SFUMKHVsv_AA/join). You can also send me a one-time payment on [Venmo](https://venmo.com/Derrick-Schultz).

## Setup

Let’s start by checking to see what GPU we’ve been assigned. Ideally we get a V100, but a P100 is fine too. Other GPUs may lead to issues.

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-03deddbf-0edc-7a18-ee7b-6aa3549b0534)


Next let’s connect our Google Drive account. This is optional but highly recommended.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install repo

The next cell will install the StyleGAN repository in Google Drive. If you have already installed it it will just move into that folder. If you don’t have Google Drive connected it will just install the necessary code in Colab.

In [3]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex torch==1.7.1 torchvision==0.8.2

/content/drive/MyDrive
/content/drive/MyDrive/colab-sg2-ada-pytorch
Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 524, done.
remote: Total 524 (delta 0), reused 0 (delta 0), pack-reused 524
Receiving objects: 100% (524/524), 8.40 MiB | 11.02 MiB/s, done.
Resolving deltas: 100% (295/295), done.
/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch
Downloading...
From: https://drive.google.com/uc?id=1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
To: /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
382MB [00:14, 26.4MB/s]
     |████████████████████████████████| 108 kB 7.5 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 12.8 MB 22 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torc

You probably don’t need to run this, but this will update your repo to the latest and greatest.

In [4]:
%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- train.py generate.py legacy.py closed_form_factorization.py flesh_digression.py apply_factor.py README.md calc_metrics.py training/stylegan2_multi.py training/training_loop.py util/utilgan.py

/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch
Already up to date.
Saved working directory and index state WIP on main: 464100c lil cleanup


## Dataset Preparation

Upload a .zip of square images to the `datasets` folder. Previously you had to convert your model to .tfrecords. That’s no longer needed :)

## Train model

Below are a series of variables you need to set to run the training. You probably won’t need to touch most of them.

* `dataset_path`: this is the path to your .zip file
* `resume_from`: if you’re starting a new dataset I recommend `'ffhq1024'` or `'./pretrained/wikiart.pkl'`
* `mirror_x` and `mirror_y`: Allow the dataset to use horizontal or vertical mirroring.

In [5]:
#required: definitely edit these!
dataset_path = './datasets/chin-morris.zip'
resume_from = './pretrained/wikiart.pkl'
aug_strength = 0.0
train_count = 0
mirror_x = True
#mirror_y = False

#optional: you might not need to edit these
gamma_value = 50.0
augs = 'bg'
config = '11gb-gpu'
snapshot_count = 4

In [6]:
!python train.py --gpus=1 --cfg=$config --metrics=None --outdir=./results --data=$dataset_path --snap=$snapshot_count --resume=$resume_from --augpipe=$augs --initstrength=$aug_strength --gamma=$gamma_value --mirror=$mirror_x --mirrory=False --nkimg=$train_count

Usage: train.py [OPTIONS]
Try 'train.py --help' for help.

Error: --data: [Errno 2] No such file or directory: './datasets/chin-morris.zip'


### Resume Training

Once Colab has shutdown, you’ll need to resume your training. Reset the variables above, particularly the `resume_from` and `aug_strength` settings.

1. Point `resume_from` to the last .pkl you trained (you’ll find these in the `results` folder)
2. Update `aug_strength` to match the augment value of the last pkl file. Often you’ll see this in the console, but you may need to look at the `log.txt`. Updating this makes sure training stays as stable as possible.
3. You may want to update `train_count` to keep track of your training progress.

Once all of this has been reset, run that variable cell and the training command cell after it.

## Convert Legacy Model

If you have an older version of a model (Tensorflow based StyleGAN, or Runway downloaded .pkl file) you’ll need to convert to the newest version. If you’ve trained in this notebook you do **not** need to use this cell.

`--source`: path to model that you want to convert

`--dest`: path and file name to convert to.

In [7]:
!python legacy.py --source=/content/drive/MyDrive/runway.pkl --dest=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/runway.pkl

Loading "/content/drive/MyDrive/runway.pkl"...
Traceback (most recent call last):
  File "legacy.py", line 382, in <module>
    convert_network_pickle() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "legacy.py", line 372, in convert_network_pickle
    with dnnlib.util.open_url(source) as f:
  File "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/dnnlib/util.py", line 389, in open_url
    return url if return_filename else open(url, "rb")
FileNotFoundError: [Errno 2] No such file or direct

## Testing/Inference

Also known as "Inference", "Evaluation" or "Testing" the model. This is the process of usinng your trained model to generate new material, usually images or videos.

### Generate Single Images

`--network`: Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`: This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation`: Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 


In [8]:
!python generate.py --outdir=/content/out/images/ --trunc=0.8 --seeds=0 --network=/content/drive/MyDrive/ladiescrop-network-snapshot-012885.pkl

Loading networks from "/content/drive/MyDrive/ladiescrop-network-snapshot-012885.pkl"...
Traceback (most recent call last):
  File "generate.py", line 492, in <module>
    generate_images() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/decorators.py", line 21, in new_func
    return f(get_current_context(), *args, **kwargs)
  File "generate.py", line 406, in generate_images
    with dnnlib.util.open_url(network_pkl) as f:
  File "/content/drive/My Drive/colab-sg2

#### Non-Square outputs

We can modify the model to output images that are not square. This isn’t as good as training a rectangular model, but with the right model it can still look nice.

* `--size` size takes in a value of `xdim-ydim`. For example, to generate a 1920x1080 image use `1920-1080`
* `--scale-type` This determines the padding style to apply in the additional space. There are four options: `pad`, `padside`, `symm`, and `symmside`. I recommend trying each one to see what works best with your images.


In [9]:
!python generate.py --outdir=/content/out/images/ --trunc=0.7 --size=1820-1024 --scale-type=symm --seeds=0-499 --network=/content/crystal.pkl

render custom size:  [1024, 1820]
padding method: symm
Loading networks from "/content/crystal.pkl"...
Traceback (most recent call last):
  File "generate.py", line 492, in <module>
    generate_images() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/decorators.py", line 21, in new_func
    return f(get_current_context(), *args, **kwargs)
  File "generate.py", line 406, in generate_images
    with dnnlib.util.open_url(network_pkl) as f:
  File "/content/drive/My D

We can use these options for any image or video generation commands (excluding projection).

### Truncation Traversal

Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

#### Options 
`--network`: Again, this should be the path to your .pkl file.

`--seeds`: Pass this only one seed. Pick a favorite from your generated images.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)


In [10]:
!python generate.py --process="truncation" --outdir=/content/out/trunc-trav-3/ --start=-0.8 --stop=2.8 --increment=0.02 --seeds=470 --network=/content/drive/MyDrive/stylegan2-transfer-models/mixed6k-network-snapshot-016470.pkl

Loading networks from "/content/drive/MyDrive/stylegan2-transfer-models/mixed6k-network-snapshot-016470.pkl"...
Traceback (most recent call last):
  File "generate.py", line 492, in <module>
    generate_images() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/decorators.py", line 21, in new_func
    return f(get_current_context(), *args, **kwargs)
  File "generate.py", line 406, in generate_images
    with dnnlib.util.open_url(network_pkl) as f:
  File "/content/d

### Interpolations

Interpolation is the process of generating very small changes to a vector in order to make it appear animated from frame to frame.

We’ll look at different examples of interpolation below.

#### Options

`--network`: path to your .pkl file

`--interpolation`: Walk type defines the type of interpolation you want. In some cases it can also specify whether you want the z space or the w space.

`--frames`: How many frames you want to produce. Use this to manage the length of your video.

`--trunc`: truncation value

#### Linear Interpolation

In [11]:
!python generate.py --outdir=/content/out/video1-w-0.5/ --space="z" --trunc=0.5 --process="interpolation" --seeds=463,470 --network=/content/drive/MyDrive/stylegan2-transfer-models/mixed6k-network-snapshot-016470.pkl

Loading networks from "/content/drive/MyDrive/stylegan2-transfer-models/mixed6k-network-snapshot-016470.pkl"...
Traceback (most recent call last):
  File "generate.py", line 492, in <module>
    generate_images() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/decorators.py", line 21, in new_func
    return f(get_current_context(), *args, **kwargs)
  File "generate.py", line 406, in generate_images
    with dnnlib.util.open_url(network_pkl) as f:
  File "/content/d

In [12]:
!python generate.py --outdir=out/video1-w/ --space="w" --trunc=1 --process="interpolation" --seeds=85,265,297,849 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

Loading networks from "/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl"...
Traceback (most recent call last):
  File "generate.py", line 492, in <module>
    generate_images() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/decorators.py", line 21, in new_func
    return f(get_current_context(), *args, **kwargs)
  File "generate.py", line 406, in generate_images
    with dnnlib.util.open_url(network_pkl) as f:
  File "/content/drive/My Drive/colab-sg2-ada-pyt

#### Slerp Interpolation

This gets a little heady, but technically linear interpolations are not the best in high-dimensional GANs. [This github link](https://github.com/soumith/dcgan.torch/issues/14) is one of the more popular explanations ad discussions.

In reality I do not find a huge difference between linear and spherical interpolations (the difference in z- and w-space is enough in many cases), but I’ve implemented slerp here for anyone interested.

In [13]:
!python generate.py --outdir=out/slerp-z/ --space="z" --trunc=1 --process="interpolation" --interpolation="slerp" --seeds=85,265,297,849 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl --frames=24

Loading networks from "/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl"...
Traceback (most recent call last):
  File "generate.py", line 492, in <module>
    generate_images() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/decorators.py", line 21, in new_func
    return f(get_current_context(), *args, **kwargs)
  File "generate.py", line 406, in generate_images
    with dnnlib.util.open_url(network_pkl) as f:
  File "/content/drive/My Drive/colab-sg2-ada-pyt

In [14]:
!python generate.py --outdir=out/slerp-w/ --space="w" --trunc=1 --process="interpolation" --interpolation="slerp" --seeds=85,265,297,849 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl --frames=12

Loading networks from "/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl"...
Traceback (most recent call last):
  File "generate.py", line 492, in <module>
    generate_images() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/decorators.py", line 21, in new_func
    return f(get_current_context(), *args, **kwargs)
  File "generate.py", line 406, in generate_images
    with dnnlib.util.open_url(network_pkl) as f:
  File "/content/drive/My Drive/colab-sg2-ada-pyt

#### Noise Loop

If you want to just make a random but fun interpolation of your model the noise loop is the way to go. It creates a random path thru the z space to show you a diverse set of images.

`--interpolation="noiseloop"`: set this to use the noise loop funtion

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--random_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

Noise loops currently only work in z space.

In [15]:
!python generate.py --outdir=out/video-noiseloop-0.9d/ --trunc=0.8 --process="interpolation" --interpolation="noiseloop" --diameter=0.9 --random_seed=100 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

Loading networks from "/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl"...
Traceback (most recent call last):
  File "generate.py", line 492, in <module>
    generate_images() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/decorators.py", line 21, in new_func
    return f(get_current_context(), *args, **kwargs)
  File "generate.py", line 406, in generate_images
    with dnnlib.util.open_url(network_pkl) as f:
  File "/content/drive/My Drive/colab-sg2-ada-pyt

#### Circular Loop

The noise loop is, well, noisy. This circular loop will feel much more even, while still providing a random loop.

I recommend using a higher `--diameter` value than you do with noise loops. Something between `50.0` and `500.0` alongside `--frames` can help control speed and diversity.

In [16]:
!python generate.py --outdir=out/video-circularloop/ --trunc=1 --process="interpolation" --interpolation="circularloop" --diameter=800.00 --frames=720 --random_seed=90 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

Loading networks from "/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl"...
Traceback (most recent call last):
  File "generate.py", line 492, in <module>
    generate_images() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/decorators.py", line 21, in new_func
    return f(get_current_context(), *args, **kwargs)
  File "generate.py", line 406, in generate_images
    with dnnlib.util.open_url(network_pkl) as f:
  File "/content/drive/My Drive/colab-sg2-ada-pyt

## Projection

### Basic Projector

*   `--target`: this is a path to the image file that you want to "find" in your model. This image must be the exact same size as your model.
*   `--num-steps`: how many iterations the projctor should run for. Lower will mean less steps and less likelihood of a good projection. Higher will take longer but will likely produce better images.



In [17]:
!python projector.py --help

Usage: projector.py [OPTIONS]

  Project given image to the latent space of
  pretrained network pickle.

  Examples:

  python projector.py --outdir=out --target=~/mytargetimg.png \
      --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl

Options:
  --network TEXT        Network pickle filename
                        [required]

  --target FILE         Target image file to
                        project to  [required]

  --num-steps INTEGER   Number of optimization
                        steps  [default: 1000]

  --seed INTEGER        Random seed  [default:
                        303]

  --save-video BOOLEAN  Save an mp4 video of
                        optimization progress
                        [default: True]

  --outdir DIR          Where to save the output
                        images  [required]

  --help                Show this message and
                        exit.


In [18]:
!python projector.py --network=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00023-chin-morris-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000304.pkl --outdir=/content/projector/ --target=/content/img005421_0.png --num-steps=200 --seed=0

Loading networks from "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00023-chin-morris-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000304.pkl"...
Traceback (most recent call last):
  File "projector.py", line 212, in <module>
    run_projection() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "projector.py", line 164, in run_projection
    with dnnlib.util.open_url(network_pkl) as fp:
  File "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/dnnlib/u

### Peter Baylies’ Projector

In [19]:
!python /content/stylegan2-ada-pytorch/pbaylies_projector.py --help

python3: can't open file '/content/stylegan2-ada-pytorch/pbaylies_projector.py': [Errno 2] No such file or directory


In [20]:
!python /content/stylegan2-ada-pytorch/pbaylies_projector.py --network=/content/ladiesblack.pkl --outdir=/content/projector-no-clip-006265-4-inv-3k/ --target-image=/content/img006265-4-inv.png --num-steps=3000 --use-clip=False --use-center=False --seed=99

python3: can't open file '/content/stylegan2-ada-pytorch/pbaylies_projector.py': [Errno 2] No such file or directory


## Combine NPZ files together

In [21]:
!python combine_npz.py --outdir=/content/npz --npzs='/content/projector-no-clip-006264-1-inv-3k/projector-no-clip-006264-1-inv-3k.npz,/content/projector-no-clip-006265-1-inv-3k/projector-no-clip-006265-1-inv-3k.npz,/content/projector-no-clip-006264-5-inv-3k/projector-no-clip-006264-5-inv-3k.npz,/content/projector-no-clip-006265-3-inv-3k/projector-no-clip-006265-3-inv-3k.npz,/content/projector-no-clip-006265-4-inv-3k/projector-no-clip-006265-4-inv-3k.npz,/content/projector-no-clip-006264-1-inv-3k/projector-no-clip-006264-1-inv-3k.npz'

Combining .npz files...
/content/projector-no-clip-006264-1-inv-3k/projector-no-clip-006264-1-inv-3k.npz
Traceback (most recent call last):
  File "combine_npz.py", line 39, in <module>
    combine_npz() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/decorators.py", line 21, in new_func
    return f(get_current_context(), *args, **kwargs)
  File "combine_npz.py", line 29, in combine_npz
    w = torch.tensor(np.load(f)['w'])
  File "/usr/local/lib/python3.7/dist-pa

In [22]:
!python generate.py --help

Usage: generate.py [OPTIONS]

  Generate images using pretrained network
  pickle.

  Examples:

  # Generate curated MetFaces images without truncation (Fig.10 left)
  python generate.py --outdir=out --trunc=1 --seeds=85,265,297,849 \
      --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl

  # Generate uncurated MetFaces images with truncation (Fig.12 upper left)
  python generate.py --outdir=out --trunc=0.7 --seeds=600-605 \
      --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl

  # Generate class conditional CIFAR-10 images (Fig.17 left, Car)
  python generate.py --outdir=out --seeds=0-35 --class=1 \
      --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/cifar10.pkl

  # Render an image from projected W
  python generate.py --outdir=out --projected_w=projected_w.npz \
      --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl

Options:
  --n

In [23]:
!python generate.py --process=interpolation --interpolation=linear --easing=easeInOutQuad --space=w --network=/content/ladiesblack.pkl --outdir=/content/combined-proj/ --projected-w=/content/npz/combined.npz --frames=120

Loading networks from "/content/ladiesblack.pkl"...
Traceback (most recent call last):
  File "generate.py", line 492, in <module>
    generate_images() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/decorators.py", line 21, in new_func
    return f(get_current_context(), *args, **kwargs)
  File "generate.py", line 406, in generate_images
    with dnnlib.util.open_url(network_pkl) as f:
  File "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/dn

## Feature Extraction using Closed Form Factorization

Feature Extraction is the process of finding “human readable” vectors in a StyleGAN model. For example, let’s say you wanted to find a vector that could open or close a mouth in a face model.

The feature extractor tries to automate the procss of finding important vectors in your model.

`--ckpt`: This is the path to your .pkl file. In other places its called `--network` (It’s a long story for why its name changed here)
`--out`: path to save your output feature vector file. The file name must end in `.pt`!

In [24]:
!python closed_form_factorization.py --out=/content/ladiesblack-cff.pt --ckpt=/content/ladiesblack.pkl

Loading networks from "/content/ladiesblack.pkl"...
Traceback (most recent call last):
  File "closed_form_factorization.py", line 25, in <module>
    with dnnlib.util.open_url(args.ckpt) as f:
  File "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/dnnlib/util.py", line 389, in open_url
    return url if return_filename else open(url, "rb")
FileNotFoundError: [Errno 2] No such file or directory: '/content/ladiesblack.pkl'


Once this cell is finished you’ll want to save that `.pt` file somewhere for reuse.

This process just created the vctor values, but we need to test it on some seed values to determine what each vector actually changes. The `apply_factor.py` script does this.

Arguments to try:


*   `-i`: This stands for index. By default, the cell above will produce 512 vectors, so `-i` can be any value from 0 to 511. I recommend starting with a higher value.
*   `-d`: This stands for degrees. This means how much change you want to see along th vector. I recommend a value between 5 and 10 to start with.
*   `--seeds`: You know what these are by now right? :)
*   `--ckpt`: path to your .pkl file
*   `--video`: adding this to your argument will produce a video that animates your seeds along the vector path. I find it much easier to figure out what’s changing with an animation.
*   `--output`: where to save the images/video
*   `--space`: By default this will use the w space to reduce entanglement

Lastly you need to add the path to the `.pt` file you made in th above cell. It’s weird, but you don’t need to add any arguments bfore it, just make sure its after `apply_factor.pt`



In [25]:
!python apply_factor.py -i 0 -d 10 --seeds 5,10 --ckpt /content/ladiesblack.pkl /content/ladiesblack-cff.pt --output /content/cff-vid/ --video

Traceback (most recent call last):
  File "apply_factor.py", line 112, in <module>
    eigvec = torch.load(args.factor)["eigvec"].to(device)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 581, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 211, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: '/content/ladiesblack-cff.pt'


That just produced images or video for a single vector, but there are 511 more! To generate every vector, you can uuse the cell below. Update any arguments you want, but don’t touch the `-i {i}` part.

**Warning:** This takes a long time, especially if you have more than one seed value (pro tip: don’t usee more than one seed value)! Also, this will take up a good amount of space in Google Drive. You’ve been warned!

In [26]:
for i in range(512):
  !python apply_factor.py -i {i} -d 10 --seeds 177 --ckpt /content/drive/MyDrive/network-snapshot-008720.pkl /content/ladies-black-cff.pt --output /content/drive/MyDrive/ladiesblack-cff-17/ --video #--out_prefix 'ladiesblack-factor-{i}'

ストリーミング出力は最後の 5000 行に切り捨てられました。
Traceback (most recent call last):
  File "apply_factor.py", line 112, in <module>
    eigvec = torch.load(args.factor)["eigvec"].to(device)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 581, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 211, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: '/content/ladies-black-cff.pt'
Traceback (most recent call last):
  File "apply_factor.py", line 112, in <module>
    eigvec = torch.load(args.factor)["eigvec"].to(device)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 581, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/py

# Layer Manipulations

The following scripts allow you to modify various resolution layers of the StyleGAN model.

## Flesh Digressions

Flesh Digressions works by manipulating the vectors in the base 4x4 layer. By doing this while leaving all the other layers untouched you can create a warping and twisting version of images from your model.

In [27]:
!python flesh_digression.py --pkl /content/stylegan2-ada-pytorch/pretrained/wikiart.pkl --psi 0.5 --seed 9999

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2392064/45929032 bytes (5.2%)6389760/45929032 bytes (13.9%)10354688/45929032 bytes (22.5%)14426112/45929032 bytes (31.4%)18423808/45929032 bytes (40.1%)22511616/45929032 bytes (49.0%)26517504/45929032 bytes (57.7%)30466048/45929032 bytes (66.3%)34365440/45929032 bytes (74.8%)38182912/45929032 bytes (83.1%)42098688/45929032 bytes (91.7%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

## Network Blending
You can take two completely different models and combine them by splitting them at a specific resolution and combining the lower layers of one model and the higher layers of another.

(Note: this tends to work best when one of the models is transfer learned from the other)

In [28]:
!python blend_models.py --lower_res_pkl /content/ffhq-pt.pkl --split_res 64 --higher_res_pkl /content/bone-bone-pt.pkl --output_path /content/ffhq-bonebone-split64.pkl

Traceback (most recent call last):
  File "blend_models.py", line 84, in <module>
    create_blended_model() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/decorators.py", line 21, in new_func
    return f(get_current_context(), *args, **kwargs)
  File "blend_models.py", line 63, in create_blended_model
    with dnnlib.util.open_url(lower_res_pkl) as f:
  File "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/dnnlib/util.py", line 389, in open_u

You can now take the output .pkl file and use that with any of the generation tools above.